By assuming to have the user's feature vectors (Each user has a 1d vector), the following code finds each users contribution in in a group of 10 users by using DPP

In [29]:
import numpy as np

def generate_user_vectors(num_users, dim):
    """
    Generate random user vectors.

    Parameters:
        num_users (int): Number of users.
        dim (int): Dimensionality of user vectors.

    Returns:
        numpy.ndarray: Array of user vectors.
    """
    user_vectors = []
    for _ in range(num_users - 1):
        user_vectors.append(np.random.rand(dim) * 0.1)  # Generate similar vectors
    user_vectors.append(np.random.rand(dim) * 10)  # Generate a different vector
    return np.array(user_vectors)

def dpp_kernel(L, subset):
    """
    Compute the DPP kernel for a given subset.

    Parameters:
        L (numpy.ndarray): The kernel matrix.
        subset (list): The subset of items.

    Returns:
        float: The probability of selecting the subset.
    """
    L_sub = L[np.ix_(subset, subset)]

    # If matrix is small, you can return the determinant by:       return np.linalg.det(L_sub).  Otherwise the following:

    """
    Compute the sign and (natural) logarithm of the determinant of an array.
    If an array has a very small or very large determinant, then a call to det may overflow or underflow.
    This routine is more robust against such issues, because it computes the logarithm of the determinant rather than the determinant itself.
    """

    return np.linalg.slogdet(L_sub)

def marginal_contribution(L, subset, user):
    """
    Compute the marginal contribution of adding a user to a subset.

    Parameters:
        L (numpy.ndarray): The kernel matrix.
        subset (list): The current subset of users.
        user (int): The user to be added.

    Returns:
        float: The marginal contribution.
    """
    # Compute the logarithm of determinant for the current subset
    sign_current, logdet_current = dpp_kernel(L, subset)

    # Compute the logarithm of determinant for the augmented subset with the user added
    sign_with_user, logdet_with_user = dpp_kernel(L, subset + [user])

    # Calculate the difference in logarithm of determinants
    delta_logdet = logdet_with_user - logdet_current

    # Return the difference
    return delta_logdet

# Number of users and dimensionality of user vectors
num_users = 4
dim = 6

# Generate user vectors
user_vectors = generate_user_vectors(num_users, dim)
print("User vectors:")
print(user_vectors)

# Compute the similarity matrix based on user vectors
similarity_matrix = np.dot(user_vectors, user_vectors.T)

# Define the DPP kernel matrix
L = similarity_matrix

# Find each user's contribution using DPP
user_contributions = []
for user in range(num_users):
    subset = list(range(user)) + list(range(user + 1, num_users))  # Exclude the current user
    contribution = marginal_contribution(L, subset, user)
    user_contributions.append(contribution)

# Print each user's contribution
for user, contribution in enumerate(user_contributions):
    print("User", user + 1, "contribution:", contribution)

User vectors:
[[0.06 0.09 0.08 0.03 0.02 0.02]
 [0.09 0.1  0.07 0.08 0.03 0.03]
 [0.05 0.08 0.1  0.07 0.06 0.09]
 [1.43 8.96 8.99 6.94 5.5  5.01]]
User 1 contribution: -6.540467593940727
User 2 contribution: -6.035833171096903
User 3 contribution: -6.013135525606293
User 4 contribution: 2.9864794893312627
